In [1]:
%matplotlib inline

In [2]:
import supervisely_lib as sly
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
team_name = 'max'
workspace_name = 'test_dtl_segmentation'
project_name = 'demo_persons'
dst_project_name = 'demo_persons_inf'
agent_name = 'max_pycharm' # None
model_name = 'yolo_coco'

In [4]:
address = os.environ['SERVER_ADDRESS']
token = os.environ['API_TOKEN']
print("Server address: ", address)
print("Your API token: ", token)

Server address:  http://192.168.1.69:5555
Your API token:  OfaV5z24gEQ7ikv2DiVdYu1CXZhMavU7POtJw2iDtQtvGUux31DUyWTXW6mZ0wd3IRuXTNtMFS9pCggewQWRcqSTUi4EJXzly8kH7MJL1hm3uZeM2MCn5HaoEYwXejKT


In [5]:
api = sly.Api(address, token)

In [6]:
team = api.team.get_info_by_name(team_name)
workspace = api.workspace.get_info_by_name(team.id, workspace_name)
print("Current context: Team {!r}, Workspace {!r}".format(team.name, workspace.name))

Current context: Team 'max', Workspace 'test_dtl_segmentation'


In [7]:
model = api.model.get_or_clone_from_explore('Supervisely/Model Zoo/YOLO v3 (COCO)', workspace.id, model_name)
print("Model: id={}, name={!r}".format(model.id, model.name))

Model: id=245, name='yolo_coco'


In [8]:
project = api.project.get_or_clone_from_explore('Supervisely/Demo/persons', workspace.id, project_name)
print("Project: id={}, name={!r}".format(project.id, project.name))

Project: id=819, name='demo_persons'


In [9]:
agent = api.agent.get_if_running(team.id, agent_name)
print("Agent: id={}, name={!r}".format(agent.id, agent.name))

Agent: id=5, name='max_pycharm'


In [10]:
task_id = api.task.deploy_model(agent.id, model.id)
print('Deploy task_id = {}'.format(task_id))

Deploy task_id = 1420


In [11]:
output_meta_json = api.model.get_output_meta(model.id)
output_meta = sly.ProjectMeta.from_json(output_meta_json)
print(output_meta)

ProjectMeta:
Object Classes
+----------------------+-----------+-----------------+
|         Name         |   Shape   |      Color      |
+----------------------+-----------+-----------------+
|     person_model     | Rectangle | [146, 208, 134] |
|    bicycle_model     | Rectangle | [116, 127, 233] |
|      car_model       | Rectangle | [233, 189, 207] |
|   motorbike_model    | Rectangle | [111, 190, 245] |
|   aeroplane_model    | Rectangle |  [92, 126, 104] |
|      bus_model       | Rectangle | [212, 239, 134] |
|     train_model      | Rectangle | [140, 180, 183] |
|     truck_model      | Rectangle | [231, 222, 180] |
|      boat_model      | Rectangle |  [213, 86, 211] |
| traffic light_model  | Rectangle | [137, 206, 104] |
|  fire hydrant_model  | Rectangle | [194, 160, 183] |
|   stop sign_model    | Rectangle | [131, 156, 191] |
| parking meter_model  | Rectangle |  [96, 163, 96]  |
|     bench_model      | Rectangle | [232, 202, 225] |
|      bird_model      | Rectangle | 

In [12]:
dst_project = api.project.create(workspace.id, dst_project_name, description="after inference", meta=output_meta.to_json(), handle_name_collision=True)
print("Destination Project: id={}, name={!r}".format(dst_project.id, dst_project.name))

Destination Project: id=823, name='demo_persons_inf_003'


In [13]:
for dataset in api.dataset.get_list(project.id):
    dst_dataset = api.dataset.create(dst_project.id, dataset.name, dataset.description, handle_name_collision=True)
    for image in api.image.get_list(dataset.id):
        dst_image = api.image.add(dst_dataset.id, image.name, image.hash, handle_name_collision=True)
        ann_json = api.model.inference_remote_image(model.id, image.hash)
        #ann = sly.Annotation.from_json(ann_json, output_meta)
        api.annotation.upload(dst_image.id, ann_json)

In [14]:
print('Inference finished. Please, project link: xxxx')

Inference finished. Please, project link: xxxx
